In [ ]:
!apt install -y -qq python3.7
!apt install -y -qq python3.8
!apt install -y -qq python3.9
!apt install -y -qq python3.10
!apt install -y -qq python3.11

In [96]:
script = """
import platform
import json
import os
import time

N = 30
n_sim = 100

def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

def bench(n):
    t1 = time.time()
    fibonacci(n)
    t2 = time.time()
    return t2-t1

try:
    with open("results.json", "r") as f:
        results = json.load(f)
except Exception:
    results = []

for _ in range(n_sim):
    duration = bench(N)
    result = {"version": platform.python_version(), "n": N, "duration": duration}
    results.append(result)

with open("results.json", "w") as f:
    json.dump(fp=f, obj=results, indent=4)
"""

with open("script.py", "w") as f:
    f.write(script)

In [97]:
!python3.7 script.py
!python3.8 script.py
!python3.9 script.py
!python3.10 script.py
!python3.11 script.py

In [151]:
latest = "3.11.0"

In [152]:
import pandas as pd
df = pd.read_json("results.json")

In [168]:
df_grouped = df[['version', 'duration']].groupby(["version"])['duration'].agg(['count','mean'])
df_grouped


latest_mean = df_grouped.loc[latest]['mean']
df_grouped['latest_improvement'] = 1 - latest_mean / df_grouped['mean']
df_grouped = df_grouped.reset_index()
df_grouped[['version_major', 'version_minor', 'version_patch']] = df_grouped['version'].str.split('.', expand=True).astype(int)
df_grouped = df_grouped.sort_values(['version_major', 'version_minor', 'version_patch'])\
    .drop(columns=['version_major', 'version_minor', 'version_patch'])\
    .reset_index(drop=True)
df_grouped

,version,count,mean,latest_improvement
0,3.7.15,100,0.337710,0.462930
1,3.8.15,100,0.350593,0.482665
2,3.9.15,100,0.353922,0.487531
3,3.10.8,100,0.347675,0.478323
4,3.11.0,100,0.181374,0.000000


In [169]:
# Python 3.11 has about 45-50% improvement over previous version for simple calculations like fibonacci function

In [170]:
df_grouped

,version,count,mean,latest_improvement
0,3.7.15,100,0.337710,0.462930
1,3.8.15,100,0.350593,0.482665
2,3.9.15,100,0.353922,0.487531
3,3.10.8,100,0.347675,0.478323
4,3.11.0,100,0.181374,0.000000


In [175]:
df_grouped.to_markdown("README.md")